In [2]:
# import zipfile as zf
# files = zf.ZipFile("/home/khoatm4/Elastic_search/News/Sample_files.zip", 'r')
# files.extractall('/home/khoatm4/Elastic_search/News/')
# files.close()

In [3]:
!pip install Elasticsearch

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import numpy as np
import pandas as pd
import os
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch('http://localhost:9200/')

In [5]:
list = os.listdir('/home/khoatm4/Elastic_search/News/News_folder')
len(list)

20072

In [6]:
path = '/home/khoatm4/Elastic_search/News/News_folder'

In [7]:
df = pd.DataFrame()
for filename in os.listdir(path):
    try:
        f = os.path.join(path, filename)
        file = open(str(f), 'r', encoding='utf-8')
        data = json.load(file)

        row = pd.json_normalize(data)
        df = pd.concat([df, row], ignore_index=True)
    except:
        print(filename)

.ipynb_checkpoints
.DS_Store


In [8]:
df['source'] = pd.Series(["vneconomy" for x in range(len(df.index))])

In [9]:
df = df[(df.Tieude != '') & (df.Body != '') & (df.Publish_time != '')]

In [10]:
(df=='').sum()

Menu             594
Tieude             0
Anh              578
Lienket            0
Motangan          46
Body               0
Publish_time       0
Received_time      0
source             0
dtype: int64

In [11]:
df

,Menu,Tieude,Anh,Lienket,Motangan,Body,Publish_time,Received_time,source
0,Tiêu điểm,Thủ tướng yêu cầu không để thiếu hụt nguồn cun...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/thu-tuong-yeu-cau-khong-d...,"Thủ tướng giao Bộ Tài chính chủ trì, phối hợp ...",Thủ tướng Chính phủ Phạm Minh Chính vừa ký ba...,1.64561e+09,1.649432e+09,vneconomy
1,Doanh nghiệp,Doanh nghiệp thủy sản “than thở” vì sắp bị thu...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/doanh-nghiep-thuy-san-tha...,Hiệp hội chế biến và xuất khẩu thủy sản Việt N...,"Theo VASEP, trong tuần qua, Hiệp hội này đã n...",1.64615e+09,1.649432e+09,vneconomy
2,Chứng khoán,3 quỹ đầu tư cùng đăng ký thoái toàn bộ vốn tạ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/3-quy-dau-tu-cung-dang-ky...,"MB Capital, JAMBVF và MBVF muốn bán toàn bộ hơ...",Sở Giao dịch Chứng khoán Tp.HCM (HOSE) thông ...,1.64934e+09,1.649431e+09,vneconomy
3,Bất động sản,Hàng ngàn hộ dân sống trong chung cư cũ thấp t...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//hang-ngan-ho-dan-song-tr...,Dự thảo Nghị định 101 được hàng chục ngàn hộ g...,Bộ Xây dựng đang lấy ý kiến lần 4 về “Dự thảo...,1.6214e+09,1.643130e+09,vneconomy
4,Chứng khoán,Nasdaq giảm hai phiên liên tiếp vì nỗi lo lạm ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//nasdaq-giam-hai-phien-li...,Cổ phiếu công nghệ và các cổ phiếu tăng trưởng...,Chỉ số Nasdaq của thị trường chứng khoán Mỹ g...,1.63774e+09,1.643129e+09,vneconomy
...,...,...,...,...,...,...,...,...,...
20064,Doanh nghiệp,Stavian Chemical thành nhà tài trợ kim cương c...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/stavian-chemical-thanh-nh...,"Ngày 23/2/2022, Ban tổ chức Đại hội thể thao Đ...",Đại hội Thể thao Đông Nam Á lần thứ 31 năm 20...,1.64563e+09,1.649432e+09,vneconomy
20066,Chứng khoán,Uỷ ban Chứng khoán ra tay xử phạt hàng loạt cổ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//uy-ban-chung-khoan-ra-ta...,Những ngày giao dịch cuối cùng của năm cũ 2021...,"Cụ thể, ngày 29/12/2021, Ủy ban Chứng khoán N...",1.64112e+09,1.643130e+09,vneconomy
20067,Chứng khoán,Thanh khoản đang tăng dần chính là nhân tố hỗ ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/thanh-khoan-dang-tang-dan...,Thị trường trong nước tăng điểm sang phiên thứ...,VnEconomy giới thiệu nhận định và khuyến nghị...,1.64444e+09,1.649431e+09,vneconomy
20068,Tiêu & Dùng,Cầu thủ Mbappé trở thành Đại sứ toàn cầu mới n...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//cau-thu-mbappe-tro-thanh...,Nhà vô địch bóng đá 22 tuổi Kylian Mbappé đã t...,Cầu thủ trẻ vốn không còn xa lạ với các thiết...,1.64011e+09,1.643129e+09,vneconomy


In [12]:
df.reset_index(inplace=True, drop=True)

In [13]:
df

,Menu,Tieude,Anh,Lienket,Motangan,Body,Publish_time,Received_time,source
0,Tiêu điểm,Thủ tướng yêu cầu không để thiếu hụt nguồn cun...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/thu-tuong-yeu-cau-khong-d...,"Thủ tướng giao Bộ Tài chính chủ trì, phối hợp ...",Thủ tướng Chính phủ Phạm Minh Chính vừa ký ba...,1.64561e+09,1.649432e+09,vneconomy
1,Doanh nghiệp,Doanh nghiệp thủy sản “than thở” vì sắp bị thu...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/doanh-nghiep-thuy-san-tha...,Hiệp hội chế biến và xuất khẩu thủy sản Việt N...,"Theo VASEP, trong tuần qua, Hiệp hội này đã n...",1.64615e+09,1.649432e+09,vneconomy
2,Chứng khoán,3 quỹ đầu tư cùng đăng ký thoái toàn bộ vốn tạ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/3-quy-dau-tu-cung-dang-ky...,"MB Capital, JAMBVF và MBVF muốn bán toàn bộ hơ...",Sở Giao dịch Chứng khoán Tp.HCM (HOSE) thông ...,1.64934e+09,1.649431e+09,vneconomy
3,Bất động sản,Hàng ngàn hộ dân sống trong chung cư cũ thấp t...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//hang-ngan-ho-dan-song-tr...,Dự thảo Nghị định 101 được hàng chục ngàn hộ g...,Bộ Xây dựng đang lấy ý kiến lần 4 về “Dự thảo...,1.6214e+09,1.643130e+09,vneconomy
4,Chứng khoán,Nasdaq giảm hai phiên liên tiếp vì nỗi lo lạm ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//nasdaq-giam-hai-phien-li...,Cổ phiếu công nghệ và các cổ phiếu tăng trưởng...,Chỉ số Nasdaq của thị trường chứng khoán Mỹ g...,1.63774e+09,1.643129e+09,vneconomy
...,...,...,...,...,...,...,...,...,...
19896,Doanh nghiệp,Stavian Chemical thành nhà tài trợ kim cương c...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/stavian-chemical-thanh-nh...,"Ngày 23/2/2022, Ban tổ chức Đại hội thể thao Đ...",Đại hội Thể thao Đông Nam Á lần thứ 31 năm 20...,1.64563e+09,1.649432e+09,vneconomy
19897,Chứng khoán,Uỷ ban Chứng khoán ra tay xử phạt hàng loạt cổ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//uy-ban-chung-khoan-ra-ta...,Những ngày giao dịch cuối cùng của năm cũ 2021...,"Cụ thể, ngày 29/12/2021, Ủy ban Chứng khoán N...",1.64112e+09,1.643130e+09,vneconomy
19898,Chứng khoán,Thanh khoản đang tăng dần chính là nhân tố hỗ ...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn/thanh-khoan-dang-tang-dan...,Thị trường trong nước tăng điểm sang phiên thứ...,VnEconomy giới thiệu nhận định và khuyến nghị...,1.64444e+09,1.649431e+09,vneconomy
19899,Tiêu & Dùng,Cầu thủ Mbappé trở thành Đại sứ toàn cầu mới n...,https://media.vneconomy.vn/w800/images/upload/...,https://vneconomy.vn//cau-thu-mbappe-tro-thanh...,Nhà vô địch bóng đá 22 tuổi Kylian Mbappé đã t...,Cầu thủ trẻ vốn không còn xa lạ với các thiết...,1.64011e+09,1.643129e+09,vneconomy


In [15]:
def doc_generator(df):
    for i in df.index:
        yield {
                "_index": 'tin_tuc_moi_nhat',
                #"_type": "_doc",
                "_id" : i,
                "_source": dict(df.iloc[i]),
            }
    raise StopIteration

helpers.bulk(es, doc_generator(df))

(19901, [])